#### Proxy Function that tracks calls made

In [1]:
from typing import List

class proxy:
    def __init__(self, edges: List[List[int]], succProb: List[float]):
        self.callCount = 0
        self.links = {}
        [
            self.links.setdefault(edges[i][0],[]).append((edges[i][1],succProb[i])) or \
            self.links.setdefault(edges[i][1],[]).append((edges[i][0],succProb[i])) \
            for i in range(len(edges))
        ]
        
    def resetCount(self):
        self.callCount = 0
        
    def neighbors(self, node: int) -> List[int]:
        self.callCount += 1
        return self.links.get(node,[])

#### Function using Dijkstra's algorithm to determine the maximum probability between starting and ending node

In [2]:
from typing import List, Callable

from heapq import heappush, heappop
def maxProbability(startNode: int, endNode: int, neighbors: Callable[[int],float]) -> [float, List]:
    probability = {startNode:1.0}
    
    # Inverting all probabilities used in the heap, as heapq is only a min priority queue and we want a max
    frontier = [(-2.0, [startNode])]
    while len(frontier) > 0:
        path = heappop(frontier)[1]
        current = path[-1]
        
        if current == endNode:
            return probability[endNode], path

        for entity in neighbors(current):
            stepProbability = probability.get(current, 0.0)*entity[1]
            if stepProbability > probability.get(entity[0], 0.0):
                probability[entity[0]] = stepProbability
                path.append(entity[0])
                heappush(frontier, (-1*stepProbability, path.copy()))
                
    return 0.0, []

#### Run a simple test case

In [3]:
edges = [[0,1],[1,2],[0,2]]
successProbability = [0.5,0.5,0.2]
startNode = 0
endNode = 2
expected = 0.25
functionProxy = proxy(edges, successProbability)

In [4]:
%%time
functionProxy.callCount = 0
probability, path = maxProbability(startNode, endNode, functionProxy.neighbors)
print(f"Function Call Count: {functionProxy.callCount}")
print(f"Valid: {expected == probability}")
print(f"Expected: {expected} : Returned: {probability}")
print(path)

Function Call Count: 2
Valid: True
Expected: 0.25 : Returned: 0.25
[0, 1, 2]
CPU times: user 316 μs, sys: 51 μs, total: 367 μs
Wall time: 312 μs


#### Create a random large graph with valid properties

In [5]:
import networkx as nx
from random import sample, random
def randomGraph(edges, nodes):
    randomGraph = nx.gnm_random_graph(edges, nodes, seed=646846841651)
    components = sorted(nx.connected_components(randomGraph), key=len, reverse=True)
    randomGraph = randomGraph.subgraph(components[0])
    successProbability = [random() for _ in range(randomGraph.number_of_edges())]
    startNode, endNode = sample(list(randomGraph.nodes),2)
    return startNode, endNode, randomGraph, successProbability

In [6]:
startNode, endNode, currentGraph, successProbability = randomGraph(10**2, 10**4)
functionProxy = proxy(list(currentGraph.edges), successProbability)

#### Run the larger random case

In [7]:
%%time
functionProxy.callCount = 0
probability, path = maxProbability(startNode, endNode, functionProxy.neighbors)
print(f"Function Call Count: {functionProxy.callCount:,}")
print(f"Returned: {probability:.4f}")
print(f"Path length: {len(path)}")

Function Call Count: 114
Returned: 0.9177
Path length: 105
CPU times: user 2.35 ms, sys: 0 ns, total: 2.35 ms
Wall time: 2.33 ms


#### Check some basic line profiling to ensure there are no unexpected hotspots

In [8]:
%load_ext line_profiler

In [9]:
%lprun -f maxProbability maxProbability(startNode, endNode, functionProxy.neighbors)

Timer unit: 1e-09 s

Total time: 0.0195024 s
File: /tmp/ipykernel_5841/2303389957.py
Function: maxProbability at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def maxProbability(startNode: int, endNode: int, neighbors: Callable[[int],float]) -> [float, List]:
     5         1       2450.0   2450.0      0.0      probability = {startNode:1.0}
     6                                               
     7                                               # Inverting all probabilities used in the heap, as heapq is only a min priority queue and we want a max
     8         1       1270.0   1270.0      0.0      frontier = [(-2.0, [startNode])]
     9       115      70409.0    612.3      0.4      while len(frontier) > 0:
    10       115     146027.0   1269.8      0.7          path = heappop(frontier)[1]
    11       115      38698.0    336.5      0.2          current = path[-1]
    12                                         

#### Confirm the function calls are scalling roughly as expected.

In [13]:
def funcCallCount(nodeCount, edgeCount):
    startNode, endNode, currentGraph, successProbability = randomGraph(nodeCount, edgeCount)
    functionProxy = proxy(list(currentGraph.edges), successProbability)
    probability, path = maxProbability(startNode, endNode, functionProxy.neighbors)
    return functionProxy.callCount

In [14]:
import numpy as np
nodeCounts = np.logspace(2,5,10)

fC = []
for nC in nodeCounts:
    fC.append(funcCallCount(int(nC), int(nC)*150))

#### Graphs are random, monotonic scaling is not expected. This is a quick check, not an exhaustive analysis.

In [15]:
print("Node Counts : Function calls")
for i in range(len(nodeCounts)):
    print(f"{nodeCounts[i]:.0}       : {fC[i]:14d}")

Node Counts : Function calls
1e+02       :             73
2e+02       :              9
5e+02       :             97
1e+03       :            585
2e+03       :           2617
5e+03       :           3475
1e+04       :           4522
2e+04       :           2264
5e+04       :          26047
1e+05       :          85382
